# Using the autoload & 1D barcode reader

| Summary | Image |
|--------|--------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"><li><b>Feature:</b> Autoload with integrated 1D barcode reader</li><li><b>Operation:</b> Automatically loads carriers and plates from the front-loading tray onto the STAR deck.</li><li><b>Barcode Recognition:</b> Scans 1D barcodes on carriers, plates, tube racks, and tip racks to confirm ID and placement.</li><li><b>Purpose:</b> Reduces manual deck verification by ensuring that physically loaded carriers match the expected layout.</li><li><b>Workflow Benefit:</b> Verification + Speeds up run setup through hands-free loading and automatic identification.</li><li><b>Best For:</b> High-throughput or frequently changing deck configurations.</li></ul> | <div style="width:320px; text-align:center;">![real_autoload](img/autoload/hamilton_star_autoload.png)<br><i>Figure: Hamilton STAR Autoload system</i></div> |


## Background: Architecture of Hamilton STAR(let) Autoload

![hamilton_star_overview](img/autoload/hamilton_autoload_overview.png)

## Setup

In [1]:
from pylabrobot.liquid_handling import LiquidHandler, STARBackend
from pylabrobot.resources import STARDeck
from pylabrobot.resources import (
  TIP_CAR_480_A00,
  PLT_CAR_L5AC_A00,
  hamilton_96_tiprack_50uL,
  Cor_96_wellplate_360ul_Fb
)

star = STARBackend()
lh = LiquidHandler(backend=star, deck=STARDeck())
await lh.setup()

# assign a tip rack carrier
tip_carrier = TIP_CAR_480_A00(name="tip_carrier")
tip_carrier[1] = tip_rack = hamilton_96_tiprack_50uL(name="tip_rack")
lh.deck.assign_child_resource(tip_carrier, rails=10)

# assign a plate carrier
plt_carrier = PLT_CAR_L5AC_A00(name="plt_carrier")
plt_carrier[0] = plate = Cor_96_wellplate_360ul_Fb(name="plt")
lh.deck.assign_child_resource(plt_carrier, rails=30)


```{note}
Two starting points are possible:
1. Carriers have been moved directly on the deck.
2. Carriers have been left on the loading tray.

Here we assume we're starting with (1) Carriers have been moved directly on the deck.
```

## Querying autoload state

In [29]:
await star.request_autoload_track()


54

In [30]:
await star.request_autoload_type()


{'cq': 0, 'id': 15}

---
## Sensing carriers

The autoload sled has a front-facing proximity sensor.
This sensor can be used to scan the entire loading tray to identify whether there are any carriers currently on the loading tray:

In [31]:
await star.request_presence_of_carriers_on_loading_tray()


[]

```{note}
The autoload detects *only* the righ-most track occupied by a carrier!
```

Similarly, if you only want to investigate a single position this can be done too. 

In [32]:
await star.request_presence_of_single_carrier_on_loading_tray(
    track=44
    )


False

The counterpart to checking for carriers on the loading tray is checking for presence of carriers on the liquid handler's deck:

In [33]:
await star.request_presence_of_carriers_on_deck()


[15, 35]

Note that we have assigned the carriers based on the left-most track they are occupying but the `request_presence_of_carriers_on_deck()` detects the right-most track of carriers:
- tip_carrier: left-most track = 10, 6 tracks wide, right-most track  15
- plt_carrier: left-most track = 30, 6 tracks wide, right-most track  35

```{note}
`.request_presence_of_carriers_on_deck()` is technically not an 'autoload' command.
It uses the presence sensors at the back of the STAR(let) deck to identify whether a carrier is present.
i.e. there is no autoload movement involved, and it therefore works for STAR(let)s without an integrated barcode sensor too.
```

Together these `STAR` methods enable capturing a full picture of the state of carriers on your liquid handler.

## Moving autoload

In [34]:
await star.move_autoload_to_track(30)


'I0XPid0019er00'

In [35]:
await star.park_autoload()


'I0XPid0020er00'

---
## Basic Load & Unload

In [36]:
await star.unload_carrier(tip_carrier, park_autoload_after=False)


'C0CRid0021er00/00'

In [37]:
await star.load_carrier(tip_carrier, park_autoload_after=False)


'C0CLid0024er00/00ci00000000bb/00'

### Command Architecture for (Un)Loading & Barcode Reading

From a firmware-perspective a carrier can be in 3 "states":
1. On the deck
2. On the autoload belt
3. On the loading tray

PyLabRobot enables easy transfer between these 3 states with these `STARBackend` methods:

![hamilton_star_overview](img/autoload/hamilton_autoload_state_transfer.png)

```{note}
We could not find a command that enables the movement sequence: deck > autoload_belt > loading_tray yet.
`.unload_carrier_after_carrier_barcode_scanning()` requires `.load_carrier_from_tray_and_scan_carrier_barcode()` to precede it.
```

## 1D Barcode reading

There are 2 main types of 1D barcodes one a classic STAR(let) deck:
1. Carrier barcodes (orientation=vertical; located at the right-back of the carrier)
2. Container barcodes:
    - TipRack barcodes (orientation=horizontal)
    - Plate barcodes (orientation=horizontal)
    - Tube barcodes (orientation=vertical)

### 1D Barcode Symbologies

All barcodes must follow a standard encoding scheme, i.e. a "symbology".
Before reading barcodes it is important to know what barcode symbology you are expecting to read!

The following barcode symbologies can be detected by the system:

In [38]:
list(star.barcode_1d_symbology_dict.keys())


['ISBT Standard',
 'Code 128 (Subset B and C)',
 'Code 39',
 'Codebar',
 'Code 2of5 Interleaved',
 'UPC A/E',
 'YESN/EAN 8',
 'Code 93']

For the highest reading safety Hamilton recommends to use barcode type `Code128 (subset B and C)`.

This is the default symbology chosen in PyLabRobot commands:

In [39]:
star._default_1d_symbology


'Code 128 (Subset B and C)'

However, you can directly set or change the expected barcode symbology:


The fastest way to read your barcode *when* your carriers are already on the deck is to move the carrier out to the identification position:

In [40]:
await star.set_1d_barcode_type("ISBT Standard")

star._default_1d_symbology


'ISBT Standard'

In [41]:
await star.set_1d_barcode_type("Code 128 (Subset B and C)")

star._default_1d_symbology


'Code 128 (Subset B and C)'

### Loading with with Barcode Reading

1D barcode reading via the autoload can only occur during carrier loading actions.
So let's first unload the carrier:

In [42]:
await star.unload_carrier(tip_carrier, park_autoload_after=False)


'C0CRid0027er00/00'

In [43]:
await star.load_carrier(
    carrier=tip_carrier,
    carrier_barcode_reading=True,
    barcode_reading=True,
    # barcode_symbology="Code 39",
    barcode_reading_direction="horizontal",
    no_container_per_carrier=5,
    park_autoload_after=False,
)


'C0CLid0031er00/00ci00000000bb/18235938752776512151/18235938752776527151/18235938752776549151/18235938752776556151/18235938752776513151'

```{warning}
The 1D barcode scanner uses a laser at a fixed height.
This is especially important when reading horizontal barcodes.
The z-height of the laser (during horizonatal) barcode reading is `z=219` or 119 mm above the deck surface.
If your 1D barcode is not precisely positioned at this height, the 1D barcode reader cannot read your barcode.
To facilitate this height, use "DWP" carriers/MFX plate_holders for plates >40 mm in `size_z`, and "MP" carriers/MFX plate_holders for plates ~15 mm in `size_z`.
```

## Advanced 1D Barcode Reading

In [44]:
await star.unload_carrier(tip_carrier, park_autoload_after=False)


'C0CRid0032er00/00'

**Loading tray -> Autoload Belt (Carrier Barcode Reading) -> Loading Tray**

In [45]:
await star.load_carrier_from_tray_and_scan_carrier_barcode(
    tip_carrier,
    # barcode_position=10,
    # barcode_reading_window_width=20.0,
    # reading_speed=50.0
    )


'C0CIid0033er00/00bb/08T0241707'

In [46]:
await star.unload_carrier_after_carrier_barcode_scanning()

'C0CAid0034er00/00'

**Loading tray -> Autoload Belt (Carrier Barcode Reading) -> Deck (Container Barcode Reading)**

-> same as simply `.load_carrier()` but split into separate components

In [47]:
await star.load_carrier_from_tray_and_scan_carrier_barcode(
    tip_carrier
    )

'C0CIid0035er00/00bb/08T0241707'

In [48]:
reading = await star.load_carrier_from_autoload_belt(
    barcode_reading=True,
    park_autoload_after=False
    )
reading

'C0CLid0037er00/00ci00000000bb/00/00/00/00/18235938752776513151'

**Deck -> Autoload Belt -> Deck (with container barcode reading only)**

In [4]:
await star.take_carrier_out_to_autoload_belt(tip_carrier)


In [5]:
reading = await star.load_carrier_from_autoload_belt(
        barcode_reading = True,
    # barcode_reading_direction = "horizontal",
    # barcode_symbology = None,
    # reading_position_of_first_barcode = 63.0,  # mm
    # no_container_per_carrier = 5,
    # distance_between_containers = 96.0,  # mm
    # width_of_reading_window = 38.0,  # mm
    # reading_speed = 128.1,  # mm/secs
    park_autoload_after=False,
    )
reading

'C0CLid0019er00/00ci00000000bb/18235938752776512151/18235938752776527151/18235938752776549151/18235938752776556151/18235938752776513151'

In [6]:
await star.park_autoload()

'I0XPid0020er00'

---
## Close Connection

In [7]:
await star.stop()